In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from plotly import graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('events.csv')

In [3]:
#퍼널 전환율 계산
funnel = pd.DataFrame(df.groupby('event_type')['user_session'].count()).reset_index()

In [4]:
funnel['stage'] = [1,2,0]

In [5]:
funnel = funnel.sort_values(by='stage').reset_index()

In [6]:
del funnel['index']

In [23]:
for i in range(len(funnel)):
    if i==0:
        funnel.loc[i, 'conversion_rate'] = None
    else:
        funnel.loc[i, 'conversion_rate'] = round(funnel.loc[i, 'user_session'] / funnel.loc[i-1, 'user_session'] *100, 2)

In [24]:
funnel

,event_type,user_session,stage,conversion_rate
0,view,793589,0,NaN
1,cart,54029,1,6.81
2,purchase,37346,2,69.12


In [28]:
fig = px.funnel(funnel, x='user_session', y='event_type', orientation='h', text='conversion_rate', width=800, height=600)
fig.show()